# **MIP2_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ig

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are the production planner in a food and beverage production fascility. You are tasked with planning production for the next 12 time periods with the goal of maximizing profit. The production fascility is able to produce two types of food and three types of beverage. The food is produced in whole units, while the beverage can be produced in any fraction of a liter. For each product that is produced at period t, a fixed cost and variable cost incurred. Each product can also be stored which is also associated with a variable cost per product stored. There is also a maximum demand for each product and time period which means that a specific product will not be sold anymore if the demand is met for that time period. Please note, that the demand does not have to be met. The revenue per product and time also differs. The cost for production and storage, the demand and revenue are all provided as in sepearte csv files named "fixed_cost_production.csv", "variable_cost_production.csv", "variable_cost_storage.csv", "demand.csv" and "revenue.csv" where all files follow the structre of the columns representing time and rows representing product. Finally, there is a constant storage capacity and production capacity for each product that resets each time period. The storage capacity is 580 units of food 1, 687 units of food 2, 599L of beverage one, 788 Liters of beverage 2 and 294L of beverage 3. The production capacity is 1080 units of food 1, 908 units of food 2, 408L of beverage one, 1000L of beverage 2 and 403L of beverage 3. """

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

1. Decision Variables:
   - X[i][t]: Quantity of the i-th product to be produced in time period t (i varies from 1 to 5 representing the 2 types of food and 3 types of beverage)
   - Y[i][t]: Quantity of the i-th product to be stored in time period t

2. Parameters:
   - FC[i]: Fixed cost of producing the i-th product (given in "fixed_cost_production.csv")
   - VCp[i]: Variable cost of producing the i-th product (given in "variable_cost_production.csv")
   - VCs[i]: Variable cost of storing the i-th product (given in "variable_cost_storage.csv")
   - D[i][t]: Demand for the i-th product in time period t (given in "demand.csv")
   - R[i][t]: Revenue from selling one unit of the i-th product in time period t (given in "revenue.csv")
   - SC[i]: Storage capacity for the i-th product (580 units for food 1, 687 units for food 2, 599L for beverage 1, 788L for beverage 2, 294L for beverage 3)
   - PC[i]: Production capacity for the i-th product (1080 units for food 1, 908 units for food 2, 40

In [9]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [10]:
print(response2.choices[0].message.content)

Objective Function:

Maximize Z = Σ(i=1 to 5) Σ(t=1 to 12) [(R[i][t]*min(X[i][t], D[i][t])) - (VCp[i]*X[i][t] + VCs[i]*Y[i][t] + FC[i]*X[i][t])] 

Here,
- The term (R[i][t]*min(X[i][t], D[i][t])) represents the revenue from selling either the produced quantity or the demand (whichever is lesser) of each product i during each time period t.
- The term (VCp[i]*X[i][t]) represents the variable production cost of each product i during each time period t.
- The term (VCs[i]*Y[i][t]) represents the variable storage cost of each product i during each time period t.
- The term (FC[i]*X[i][t]) represents the fixed production cost of each product i during each time period t. 

The objective is to maximize the total profit across all product types and time periods. The first terms within the parentheses refer to the revenue while the remaining terms refer to the associated costs of production and storage.


In [11]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [12]:
print(response3.choices[0].message.content)

1. Production Quantity Constraints: The quantity of each product produced in each time period must not exceed the production capacity for that product.
   - X[i][t] <= PC[i] for all i and t
   
2. Demand Constraints: The quantity of each product produced in each time period must not exceed the demand for that product in that time period.
   - X[i][t] <= D[i][t] for all i and t

3. Storage Capacity Constraints: The quantity of each product stored in each time period must not exceed the storage capacity for that product.
   - Y[i][t] <= SC[i] for all i and t

4. Continuity Constraints: The quantity of each product produced plus the quantity stored from the previous time period should be equal to the quantity sold plus the quantity stored in the current period under consideration. This ensures that supply meets the demand after considering the storage. 
   - X[i][t] + Y[i][t-1] = min(X[i][t], D[i][t]) + Y[i][t] for all i and t (except t=1 where Y[i][0] = 0)

5. Non-negativity Constraints:

### **Generate Pyomo Code**

In [13]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [14]:
Markdown(response4.choices[0].message.content)

Since a complete python script with data retrieval using csv files is too large to provide, the following example is a simplified version of the problem using only 2 products and 4 time periods and only uses python lists for the data storage instead of csv files.

```python
from pyomo.environ import *

#Initialize the model
model = ConcreteModel()

#Define the sets: i=2 products, t=4 periods
model.i = Set(initialize=[1,2], doc='Products')
model.t = Set(initialize=[1,2,3,4], doc='Time periods')

params = {'FC': [50,60], 'VCp': [10,12], 'VCs':[5,6], 'D': [[100,120,140,160],[200,220,240,260]], 'R': [[20,22,24,26],[30,32,34,36]], 'SC': [580,687], 'PC': [1080,908]}
#Sample data

#Parameters
model.FC = Param(model.i, initialize={1:params['FC'][0], 2:params['FC'][1]}, doc='Fixed production cost')
model.VCp = Param(model.i, initialize={1:params['VCp'][0], 2:params['VCp'][1]}, doc='Variable production cost')
model.VCs = Param(model.i, initialize={1:params['VCs'][0], 2:params['VCs'][1]}, doc='Variable storage cost')
model.D = Param(model.i, model.t, initialize={(1,1):params['D'][0][0], (1,2):params['D'][0][1], (1,3):params['D'][0][2], (1,4):params['D'][0][3], (2,1):params['D'][1][0], (2,2):params['D'][1][1], (2,3):params['D'][1][2], (2,4):params['D'][1][3]}, doc='Demand')
model.R = Param(model.i, model.t, initialize={(1,1):params['R'][0][0], (1,2):params['R'][0][1], (1,3):params['R'][0][2], (1,4):params['R'][0][3], (2,1):params['R'][1][0], (2,2):params['R'][1][1], (2,3):params['R'][1][2], (2,4):params['R'][1][3]}, doc='Revenue')
model.SC = Param(model.i, initialize={1:params['SC'][0], 2:params['SC'][1]}, doc='Storage Capacity')
model.PC = Param(model.i, initialize={1:params['PC'][0], 2:params['PC'][1]}, doc='Production Capacity')

#Variables
model.X = Var(model.i, model.t, bounds=(0.0,None), doc='Production')
model.Y = Var(model.i, model.t, bounds=(0.0,None), doc='Storage')

#Objective
def obj_rule(model):
    return sum(sum(model.R[i,t]*min(model.X[i,t],model.D[i,t]) - model.VCp[i]*model.X[i,t] - model.VCs[i]*model.Y[i,t] - model.FC[i]*model.X[i,t] for t in model.t) for i in model.i)
model.obj = Objective(rule=obj_rule, sense=maximize, doc='Objective function')

#Constraints
def demand_rule(model, i, t):
    return model.X[i,t] <= model.D[i,t]
model.demand = Constraint(model.i, model.t, rule=demand_rule, doc='Demand')

def production_rule(model, i, t):
    return model.X[i,t] <= model.PC[i]
model.production = Constraint(model.i, model.t, rule=production_rule, doc='Production')

def storage_rule(model, i, t):
    return model.Y[i,t] <= model.SC[i]
model.storage = Constraint(model.i, model.t, rule=storage_rule, doc='Storage')

def continuity_rule(model, i, t):
    if t==1:
        return model.X[i,1] == min(model.X[i,t],model.D[i,t]) + model.Y[i,1]
    else:
        return model.X[i,t] + model.Y[i,t-1] == min(model.X[i,t],model.D[i,t]) + model.Y[i,t]
model.continuity = Constraint(model.i, model.t, rule=continuity_rule, doc='Continuity')

# Solve the problem
solver = SolverFactory('glpk')
solver.solve(model)
```

### **Run the code Generated by GPT4**

In [15]:
from pyomo.environ import *

# Initialize the model
model = ConcreteModel()

# Define the sets: i=2 products, t=4 periods
model.i = Set(initialize=[1,2], doc='Products')
model.t = Set(initialize=[1,2,3,4], doc='Time periods')

params = {'FC': [50,60], 'VCp': [10,12], 'VCs':[5,6], 'D': [[100,120,140,160],[200,220,240,260]], 'R': [[20,22,24,26],[30,32,34,36]], 'SC': [580,687], 'PC': [1080,908]}
# Sample data

# Parameters
model.FC = Param(model.i, initialize={1:params['FC'][0], 2:params['FC'][1]}, doc='Fixed production cost')
model.VCp = Param(model.i, initialize={1:params['VCp'][0], 2:params['VCp'][1]}, doc='Variable production cost')
model.VCs = Param(model.i, initialize={1:params['VCs'][0], 2:params['VCs'][1]}, doc='Variable storage cost')
model.D = Param(model.i, model.t, initialize={(1,1):params['D'][0][0], (1,2):params['D'][0][1], (1,3):params['D'][0][2], (1,4):params['D'][0][3], (2,1):params['D'][1][0], (2,2):params['D'][1][1], (2,3):params['D'][1][2], (2,4):params['D'][1][3]}, doc='Demand')
model.R = Param(model.i, model.t, initialize={(1,1):params['R'][0][0], (1,2):params['R'][0][1], (1,3):params['R'][0][2], (1,4):params['R'][0][3], (2,1):params['R'][1][0], (2,2):params['R'][1][1], (2,3):params['R'][1][2], (2,4):params['R'][1][3]}, doc='Revenue')
model.SC = Param(model.i, initialize={1:params['SC'][0], 2:params['SC'][1]}, doc='Storage Capacity')
model.PC = Param(model.i, initialize={1:params['PC'][0], 2:params['PC'][1]}, doc='Production Capacity')

# Variables
model.X = Var(model.i, model.t, bounds=(0.0,None), doc='Production')
model.Y = Var(model.i, model.t, bounds=(0.0,None), doc='Storage')

# Objective
def obj_rule(model):
    return sum(sum(model.R[i,t]*min(model.X[i,t],model.D[i,t]) - model.VCp[i]*model.X[i,t] - model.VCs[i]*model.Y[i,t] - model.FC[i]*model.X[i,t] for t in model.t) for i in model.i)
model.obj = Objective(rule=obj_rule, sense=maximize, doc='Objective function')

# Constraints
def demand_rule(model, i, t):
    return model.X[i,t] <= model.D[i,t]
model.demand = Constraint(model.i, model.t, rule=demand_rule, doc='Demand')

def production_rule(model, i, t):
    return model.X[i,t] <= model.PC[i]
model.production = Constraint(model.i, model.t, rule=production_rule, doc='Production')

def storage_rule(model, i, t):
    return model.Y[i,t] <= model.SC[i]
model.storage = Constraint(model.i, model.t, rule=storage_rule, doc='Storage')

def continuity_rule(model, i, t):
    if t==1:
        return model.X[i,1] == min(model.X[i,t],model.D[i,t]) + model.Y[i,1]
    else:
        return model.X[i,t] + model.Y[i,t-1] == min(model.X[i,t],model.D[i,t]) + model.Y[i,t]
model.continuity = Constraint(model.i, model.t, rule=continuity_rule, doc='Continuity')

# Solve the problem
solver = SolverFactory('glpk')
solver.solve(model)

ERROR:pyomo.core:Rule failed when generating expression for Objective obj with index None:
PyomoException: Cannot convert non-constant Pyomo expression (100  <  X[1,1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.
ERROR:pyomo.core:Constructing component 'obj' from data=None failed:
    PyomoException: Cannot convert non-constant Pyomo expression (100  <  X[1,1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both 

PyomoException: Cannot convert non-constant Pyomo expression (100  <  X[1,1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**